# Transfer learning

## 0. Import packages and modules

In [1]:
import os
# Limit log messages from Tensorflow
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import pathlib

import sklearn
import tensorflow
import keras

print(sklearn.__version__)
print(tensorflow.__version__)
print(keras.__version__)

1.6.1
2.18.1
3.9.2


In [3]:
# print GPU info

print(tensorflow.config.list_physical_devices('GPU'))

print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


## 1. Formulate / Outline the problem: Image classification

In [4]:
DATA_FOLDER = pathlib.Path('/scratch/project_465001310/env-deep-learning-intro/data/dataset_dollarstreet/') # change to location where you stored the data
train_images = np.load(DATA_FOLDER / 'train_images.npy')
val_images = np.load(DATA_FOLDER / 'test_images.npy')
train_labels = np.load(DATA_FOLDER / 'train_labels.npy')
val_labels = np.load(DATA_FOLDER / 'test_labels.npy')

## 2. Identify inputs and outputs

## 3. Prepare the data

In [5]:
train_images = train_images / 255.0
val_images = val_images / 255.0

## 4. Choose a pre-trained model or start building architecture from scratch

In [6]:
# input tensor
inputs = keras.Input(train_images.shape[1:])

In [8]:
# upscale layer
import tensorflow as tf

method = tf.image.ResizeMethod.BILINEAR
upscale = keras.layers.Lambda(
  lambda x: tf.image.resize_with_pad(x, 160, 160, method=method))(inputs)

In [9]:
base_model = keras.applications.DenseNet121(include_top=False,
                                            pooling='max',
                                            weights='imagenet',
                                            input_tensor=upscale,
                                            input_shape=(160,160,3),
                                            )

I0000 00:00:1746569456.776149   56452 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 63718 MB memory:  -> device: 0, name: AMD Instinct MI250X, pci bus id: 0000:d9:00.0


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


### Only train a ‘head’ network

In [10]:
base_model.trainable = False

In [11]:
out = base_model.output
out = keras.layers.Flatten()(out)
out = keras.layers.BatchNormalization()(out)
out = keras.layers.Dense(50, activation='relu')(out)
out = keras.layers.Dropout(0.5)(out)
out = keras.layers.Dense(10)(out)

In [12]:
model = keras.models.Model(inputs=inputs, outputs=out)

## Exercise

### Replace all `TODO`

In [ ]:
# compile the model
model.compile(optimizer=...,  # TODO: string
              loss=...,  # TODO: loss function class from keras.losses module
              metrics=['accuracy'])


In [ ]:
# train the model
early_stopper = ...  # TODO: earlystopper class from keras.callbacks module with a patience of 5

history = model.fit(x=train_images,
                    y=train_labels,
                    batch_size=...,  # TODO: int
                    epochs=...,  # TODO: int
                    callbacks=[early_stopper],
                    validation_data=(val_images, val_labels))

In [ ]:
# inspect the results

def plot_history(history, metrics):
    """
    Plot the training history

    Args:
        history (keras History object that is returned by model.fit())
        metrics(str, list): Metric or a list of metrics to plot
    """
    history_df = pd.DataFrame.from_dict(history.history)
    sns.lineplot(data=history_df[metrics])
    plt.xlabel("epochs")
    plt.ylabel("metric")

plot_history(history, [..., ...]) # TODO: choose the metric

## Conclusion